<a href="https://colab.research.google.com/github/AmineSdk/RecommenderSystem/blob/main/CF_Social_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-text

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aminesedkaoui/yelp-user-user-trust
! kaggle datasets download -d aminesedkaoui/yelp-review-ratings

  0% 0.00/40.1k [00:00<?, ?B/s]
100% 40.1k/40.1k [00:00<00:00, 28.0MB/s]
 49% 25.0M/51.5M [00:00<00:00, 52.4MB/s]
100% 51.5M/51.5M [00:00<00:00, 78.0MB/s]


In [3]:
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [4]:
!unzip "/content/yelp-user-user-trust.zip" -d "/content/"
!unzip "/content/yelp-review-ratings.zip" -d "/content/"

Archive:  /content/yelp-user-user-trust.zip
  inflating: /content/yelp_user_user_trust.csv  
Archive:  /content/yelp-review-ratings.zip
  inflating: /content/yelp_review_ratings.csv  


In [5]:
df_user_user = pd.read_csv('/content/yelp_user_user_trust.csv')
df_rating = pd.read_csv('/content/yelp_review_ratings.csv')
df_rating.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'itemID', 'user_id': 'userID'}, inplace = True)
df_rating = df_rating.drop(['Unnamed: 0'],axis=1)
df_rating.head()

,userID,itemID,rating,reviewText
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo..."
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...


In [6]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item,df_user_item

In [33]:
def getUsersRatings(df,usersList):
  temp = df.copy()
  for user in list(df['userID']):
    if user not in usersList:
      temp = temp.drop(temp.loc[temp['userID'] == user].index)
  return temp

In [87]:
def genIndexColumn(df,min_index,max_index,min_column,max_column):
  New_User_IDs = random.sample(range(min_index,max_index),df.index.size)
  New_Item_IDs = random.sample(range(min_column,max_column),df.columns.size)

  df_new = pd.DataFrame(df.to_numpy(),index = New_User_IDs,columns = New_Item_IDs)
  return df_new

In [8]:
def generateIDs(df,index,columns,min_index,max_index,min_column,max_column):
  users = df[index].unique()
  items = df[columns].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(min_index,max_index),df[index].nunique())
  New_Item_IDs = random.sample(range(min_column,max_column),df[columns].nunique())
  i = 0
  for d in users :
    df_train[index].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in items :
    df_train[columns].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  return df_train

In [9]:
def preprocess_df(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id):
  
  df_train = generateIDs(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id)
  
  df_train['one']=df_train['rating'].apply(lambda x: 1 if x==1 else 0)
  df_train['two']=df_train['rating'].apply(lambda x: 1 if x==2 else 0)
  df_train['three']=df_train['rating'].apply(lambda x: 1 if x==3 else 0)
  df_train['four']=df_train['rating'].apply(lambda x: 1 if x==4 else 0)
  df_train['five']=df_train['rating'].apply(lambda x: 1 if x==5 else 0)
  df_train['six']=df_train['rating'].apply(lambda x: 1 if x==6 else 0)

  return df_train

In [25]:
from numpy.core.fromnumeric import size
#GMF

def Create_GMF(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(6, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF

def Train_GMF(model,X_train,Y_train,nbrEpochs):
  model.compile(optimizer = 'adam',
                    loss = tf.keras.losses.categorical_crossentropy ,
                    metrics=['accuracy'])
  model.fit(X_train,Y_train,epochs = nbrEpochs)
  return model

def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()
  for user in userIDs:
    temp = pd.Series(itemIDs)
    item_s = item_s.append(temp)
    temp = []
    temp = [user for item in itemIDs]
    temp = pd.Series(temp)
    user_s = user_s.append(temp)
    
    progress = user_s.size*100/(userIDs.size * itemIDs.size)
   
    if (progress  > 10) & (int(progress) < 25) & (i == 0):
      print("===== 10 % =====")
      i += 1
    elif (progress > 25) & (int(progress) < 35) & (i == 1):
      print("===== 25 % =====")
      i += 1
    elif (progress > 35) & (int(progress) < 50) & (i == 2):
      print("===== 35 % =====")
      i += 1
    elif (progress > 50) & (int(progress) < 65) & (i == 3):
      print("===== 50 % =====")
      i += 1
    elif (progress > 65) & (int(progress) < 75) & (i == 4):
      print("===== 65 % =====")
      i += 1
    elif (progress > 75) & (int(progress) < 80) & (i == 5):
      print("===== 75 % =====")
      i += 1
    elif (progress > 80) & (int(progress) < 90) & (i == 6):
      print("===== 80 % =====")
      i += 1
    elif (progress > 90) & (int(progress) < 95) & (i == 7):
      print("===== 90 % =====")
      i += 1
    elif (progress > 95) & (i == 8):
      print("===== 95 % =====")
      i += 1

  return user_s,item_s

def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):

  prediction = model.predict([userList,itemList],verbose = 0)
  i = 0
  row = []
  matrix = []
  print("pred done")

  while i < userList.shape[0]:
    result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    row.append(result)
    if len(row) == itemIDs.size:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
  
  return dataframe


In [133]:
df_train = df_rating[:5000]
df_train['userID'].nunique()

4843

In [134]:
df_train['itemID'].nunique()

2663

In [136]:
df_train_new = getUsersRatings(df_train,list(df_user_user.index))

In [137]:
df_train = preprocess_df(df_train_new,'userID','itemID',5000,9999,20000,30000)

In [52]:
df_user_user.set_index(['Unnamed: 0'],inplace=True)

In [ ]:
df_user_user

In [ ]:
df_train

In [120]:
mat = []
for user in df_user_user.index:
  mat.append(list(df_mat.loc[user]))


df_train_r = pd.DataFrame(mat,index= df_user_user.index,columns = df_mat.columns)

In [ ]:
df_train['userID']

In [129]:
df_train_r = genIndexColumn(df_train_r,5000,9999,20000,30000)

In [ ]:
df_train_r

In [101]:
GMF = Create_GMF(df_train['userID'].max() + 1,df_train['itemID'].max() + 1,64)

In [103]:
df_train_sample = df_train
X_train, X_test,y_train,y_test= train_test_split(df_train_sample[['userID','itemID']],df_train_sample[['one','two','three','four','five','six']],stratify=df_train_sample[['one','two','three','four','five','six']])
#X_train, X_test,y_train,y_test= train_test_split(df_train[['User_ID','Item_ID']],df_train['rating'],stratify=df_train['rating'])

In [117]:
GMF_trained = Train_GMF(GMF,[X_train['userID'],X_train['itemID']],y_train,5)

Epoch 1/5
16/16 [==============================] - 2s 31ms/step - loss: 0.0293 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 1s 36ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 1s 39ms/step - loss: 0.0203 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 1s 45ms/step - loss: 0.0171 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 1s 43ms/step - loss: 0.0146 - accuracy: 1.0000


In [118]:
GMF_trained.evaluate([X_test['userID'],X_test['itemID']],y_test)

6/6 [==============================] - 0s 6ms/step - loss: 1.4787 - accuracy: 0.4136


[1.478668451309204, 0.4135802388191223]

In [119]:
user_item_matrix , df_mat = DatasetToUserItemDataFrame(df_train_new,'userID','itemID','rating')

In [121]:
New_User_IDs = df_train_r.index
New_Item_IDs = df_train_r.columns
New_Item_IDs.size * New_User_IDs.size

328160

In [ ]:
user_s,item_s = user_item_ID_lists( df_train['userID'],df_train['itemID'])

In [ ]:
df_mat_filled = Fill_Cf_Matrix(GMF_trained,user_s,item_s,New_User_IDs,New_Item_IDs)

In [ ]:
df_mat_filled.head()

In [ ]:
df_mat.head()

In [60]:
df_u_u = df_user_user.drop(['Unnamed: 0'],axis=1)

In [ ]:
df_u_u

In [61]:
df_arr = df_u_u.to_numpy()

In [ ]:
df_mat_trust = df_arr

In [54]:
df_mat_sample =df_mat_filled

In [ ]:
df_mat_sample.head()

In [ ]:
def to_binary(x):
  if (x > 0):
    x = 1
  return x

In [ ]:
def df_to_binary(df):
  for col in df.columns:
    df[col] = df[col].apply(to_binary)
  return df

In [56]:
#Social_Autoencoder

rating_input = layers.Input(shape=(df_mat_sample.shape[1]),name='user_item')
social_input = layers.Input(shape=(df_mat_sample.shape[1]),name='user_user')

flat_rating = layers.Flatten()(rating_input)
flat_social = layers.Flatten()(social_input)

#dropout = layers.Dropout(.2)(flat)
SharedLayer_encoder = layers.Concatenate()([flat_rating,flat_social])
hid_encoder = layers.Dense(256,activation="relu")(SharedLayer_encoder)
# hid_encoder1 = layers.Dense(256,activation="relu")(hid_encoder)
# hid_encoder2 = layers.Dense(128,activation="relu")(hid_encoder1)
# hid_encoder3 = layers.Dense(10,activation="relu")(hid_encoder2)
encoder_output = layers.Dense(128,activation="relu")(hid_encoder)

SharedLayer_decoder = layers.Dense(256,activation="relu")(encoder_output)
# decoder_hidden1 = layers.Dense(256,activation="relu")(decoder_input)
# decoder_hidden2 = layers.Dense(512,activation="relu")(decoder_hidden1)
# decoder_hidden3 = layers.Dense(25,activation="relu")(decoder_hidden2)
rating_decoded , social_decoded = tf.split(SharedLayer_decoder,[df_mat_sample.shape[1],df_mat_sample.shape[1]])

rating_output = layers.Dense(df_mat_sample.shape[1],activation="relu")(rating_decoded)
social_output = layers.Dense(df_mat_sample.shape[1],activation="relu")(social_decoded)

autoencoder = tf.keras.Model(inputs = [rating_input,social_input], outputs = [rating_output,social_output])

In [ ]:
def rmse (y_true,y_pred):
  y_pred = tf.cast(y_pred, tf.float32)
  y_true = tf.cast(y_true, tf.float32)
  rmse = y_true - y_pred
  rmse = tf.square(rmse)
  rmse = tf.math.reduce_mean(rmse)
  rmse = tf.math.sqrt(rmse)
  return rmse

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

In [ ]:
autoencoder.compile(optimizer ='adam',
                    loss = tf.keras.losses.MeanAbsoluteError(),
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_mat,df_mat_sample)

In [ ]:
X_train

In [ ]:
autoencoder.fit(X_train, y_train, epochs = 2000)

In [ ]:
autoencoder.evaluate(X_test,y_test)

26/26 [==============================] - 0s 3ms/step - loss: 0.3342 - accuracy: 0.0012


[0.33423539996147156, 0.0012330455938354135]

In [ ]:
df_mat

In [ ]:
mat_pred = autoencoder.predict(df_mat)

102/102 [==============================] - 0s 2ms/step


In [ ]:
mat_pred

In [ ]:
df_mat_pred = pd.DataFrame(mat_pred, index = df_mat_sample.index, columns = df_mat_sample.columns)

In [ ]:
df_mat_pred.max()

189722    6.472934
122571    6.492408
100211    6.818763
101960    6.571887
105337    6.085056
            ...   
137594    5.982568
198330    7.050071
136306    6.202019
197491    7.210152
124494    5.497391
Length: 1369, dtype: float32

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
d = scaler.fit_transform(df_mat_pred)
scaled_df = pd.DataFrame(d,index = df_mat_pred.index,columns = df_mat_pred.columns)
scaled_df

In [ ]:
for col in df_mat_pred.columns:
 df_mat_pred[col] = df_mat_pred[col].apply(lambda x: 5 if x> 5.5 else x)

In [ ]:
df_mat_pred

In [ ]:
res = np.where(df_mat_pred.iloc[:,:].values == 1 )
res[0].size